In [1]:
import ee
import geemap
import hvplot.pandas
import hvplot.xarray
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
ee.Authenticate()

Exception: Problem requesting tokens. Please try again.  HTTP Error 400: Bad Request b'{\n  "error": "invalid_request",\n  "error_description": "Missing required parameter: code"\n}'

In [5]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [4]:
import json
import urllib.request

def get_mrc_metadata(return_gdf=True, verbose=False):
    """ Get Metadata from Mekong River Comission Data Portal. """
    url = r'https://api.mrcmekong.org/api/v1/ts/inventory/timeSeriesList'
    urllib.request.urlretrieve(url, 'timeSeriesList.json')
    
    if verbose:
        print(f'Downloaded time-series metadata from {url} .')
    
    f = open('timeSeriesList.json')
    data = json.load(f)
    df_metadata = pd.DataFrame([])
    for dataset in data:
        df_temp = pd.DataFrame([dict(dataset)])
        df_temp['longitude'] = df_temp['longitude'].astype(float)
        df_temp['latitude'] = df_temp['latitude'].astype(float)
        df_metadata = pd.concat([df_temp, df_metadata])
        f.close()
    df_metadata = df_metadata.reset_index().drop(columns=['index'])
    
    if verbose:
        print(f'Found a total of {df_metadata.shape[0]} time-series datasets from {len(df_metadata.stationCode.unique())} stations of the MRC Data Portal.')
        
    if return_gdf:
        gdf_metadata = gpd.GeoDataFrame(
            df_metadata, geometry=gpd.points_from_xy(df_metadata.longitude, df_metadata.latitude), crs="EPSG:4326"
        )
        return(gdf_metadata)
    else:
        return(df_metadata)

gdf_metadata = get_mrc_metadata(return_gdf=True)
gdf_metadata_dmsp = gdf_metadata.loc[gdf_metadata.label.str.contains('DSMP')]
gdf_stations_dsmp = gdf_metadata_dmsp.groupby('locationIdentifier').first()[['river', 'stationShortName', 'geometry']].set_crs('EPSG:4326')
Map.add_gdf(gdf_stations_dsmp, 'MRC DSMP stations', {'color': 'red'})

Exception: ee_object must be one of ee.Geometry, ee.FeatureCollection, ee.Image, or ee.ImageCollection.

In [ ]:
gdf_q = gdf_metadata.loc[(gdf_metadata.label.isin(['Daily Discharge', 'Daily Calculated'])) & (gdf_metadata.locationIdentifier.isin(gdf_stations_dsmp.index))]

paths_data_measured = list(Path(f'outputs/csv/').rglob(f'*MeasuredDSMP*.csv'))
paths_data_calculated = list(Path(f'outputs/csv/').rglob(f'*CalculatedDSMP*.csv'))
df_data_dsmp = pd.DataFrame([])
for path in paths_data_measured+paths_data_calculated:
    df_temp = pd.read_csv(path, dtype={'station_code':'str'})
    df_temp['date'] = pd.to_datetime(df_temp['date'])
    df_temp['med_frq'] = np.median(np.diff(df_temp.date))
    df_data_dsmp = pd.concat([df_data_dsmp, df_temp])

# Create geodataframe
gdf_metadata_dsmp = gpd.GeoDataFrame(df_metadata_subset_DSMP,
                       crs={'init': 'epsg:4326'},
                       geometry=df_metadata_subset_DSMP.apply(lambda row: shapely.geometry.Point((row.longitude, row.latitude)), axis=1)).reset_index()

In [ ]:
paths_data_measured = list(Path(f'outputs/csv/').rglob(f'*DailyDischarge*.csv'))
paths_data_calculated = list(Path(f'outputs/csv/').rglob(f'*DailyCalculated*.csv'))
df_data_dsmp_q = pd.DataFrame([])
for path in paths_data_measured+paths_data_calculated:
    df_temp = pd.read_csv(path, dtype={'station_code':'str'})
    df_temp['date'] = pd.to_datetime(df_temp['date'])
    df_temp['med_frq'] = np.median(np.diff(df_temp.date))
    df_data_dsmp_q = pd.concat([df_data_dsmp_q, df_temp])

df_data_dsmp_q

In [ ]:
transect = Map.draw_features[0].buffer(45)
Map.addLayer(transect, {'color': 'red'}, 'Transect (test)')

In [ ]:
kwargs = {
}
# Set timespan
start_date, end_date = '2000-01-01', '2024-01-30'

# Cloud masking (scene-based)
cld_filt_thresh = 80        # Maximum image cloud cover percent allowed in image collection

# water masking
mask_water = True

# # Cloud masking (pixel-based, s2cloudless only)
# cld_prb_thresh = 25      # Cloud probability (%); values greater than are considered cloud
# cld_prj_dist = 1          # Maximum distance (km) to search for cloud shadows from cloud edges (based on Hollstein decision tree)

# Cloud masking (pixel-based, cloud score+ only)
qa_band = 'cs_cdf'
clear_thresh = 0.75

In [ ]:
# load 3S basin
json_data = 'geometries/geoms.geojson'
fc_geoms = geemap.geojson_to_ee(json_data)
roi_geom = fc_geoms.first().geometry()

# load dams
df = pd.read_csv('geometries/3SReservoirs.csv')
gdf_dams = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_xy(df['X'], df['Y']), crs=4326).drop(columns=['X', 'Y']).set_index('id')
fc_dams = geemap.geopandas_to_ee(gdf_dams)

# define stations
stations = ee.FeatureCollection([
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([106.39220569493087, 14.11961739545873]), {'station_id': '430102', 'station_name': 'Siempang'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([107.78237137912318, 12.897991717109619]), {'station_id': '451305', 'station_name': 'Ban Don'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([106.5278998223061, 13.553039642985812]), {'station_id': '450101', 'station_name': 'Lum Phat'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([107.47041987706022, 13.940093424442852]), {'station_id': '440202', 'station_name': 'Pleiku'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([107.10639453530615, 14.050489408398247]), {'station_id': '440103', 'station_name': 'Andaung Meas'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([107.44760623609672, 13.792268298200513]), {'station_id': '440100', 'station_name': 'Phum Pi'}),
  ee.Feature(ee.Algorithms.GeometryConstructors.Point([105.987503299606, 13.537303972542501]), {'station_id': '014501', 'station_name': 'Stung Treng'}),
])

stations = ee.FeatureCollection([
  ee.Feature(transect, {'station_id': '430102', 'station_name': 'Siempang'})
])

lake_rhone = ee.Feature(ee.FeatureCollection('users/michaelbrechbuehler/Landsat_ST/shapefiles/rhone_wgs84').first().geometry().centroid(), {'station_id': 'Rhonesee'})
lake_steisee = ee.Feature(ee.FeatureCollection('users/michaelbrechbuehler/Landsat_ST/shapefiles/steisee_wgs84').first().geometry().centroid(), {'station_id': 'Steinsee'})

stations = transect

# # add S2 clear median
# filters = ee.Filter.And(
#     ee.Filter.bounds(roi_geom), 
#     ee.Filter.date('2017-12-01', '2019-03-31'),
#     ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)
# )
# s2_median = ee.ImageCollection("COPERNICUS/S2_SR").filter(filters).median()

# Map.addLayer(s2_median, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}, 'Sentinel-2 RGB')
#Map.addLayer(roi_geom, {'color': 'yellow'}, '3S Basin')
Map.addLayer(stations, {'color': 'red'}, 'Stations')
Map.add_gdf(gdf_dams, 'Dams', {'color': 'blue'})
Map.centerObject(stations)

## Get point samples at virtual stations

In [ ]:
stations = stations.filter(ee.Filter.eq('station_name', 'Lum Phat'))

In [ ]:
stations

In [ ]:
%load_ext autoreload
%autoreload 2
import functions_process as funcs_process
import functions_turbidity as funcs_turb
import functions_sampling as funcs_sampling

for idx, row in geemap.ee_to_df(stations).iterrows():
    name, id = row.station_name, row.station_id
    print(f'{name} ({id})')
    station = stations.filter(ee.Filter.eq('station_id', id))

    # get Rrs imagecolls
    ic_msi = funcs_process.load_rrs_imcoll(sensor='msi', start_date=start_date, end_date=end_date, mask_water=mask_water, bounds=station)
    ic_oli = funcs_process.load_rrs_imcoll(sensor='oli', start_date=start_date, end_date=end_date, mask_water=mask_water, bounds=station)
    ic_etm = funcs_process.load_rrs_imcoll(sensor='etm', start_date=start_date, end_date=end_date, mask_water=mask_water, bounds=station)
    ic_all = ic_msi.merge(ic_oli).merge(ic_etm)
    
    # comput tsm features
    ic_all = ic_all \
        .map(funcs_turb.calc_spm_nechad) \
        .map(funcs_turb.calc_tur_nechad) \
        .map(funcs_turb.calc_tur_dogliotti) \
        .map(funcs_turb.calc_indices)

    # merge imagecolls and sample
    fc_all = ee.FeatureCollection(ic_all.map(funcs_sampling.sample_image(station))).flatten()

    # export to drive
    geemap.ee_export_vector_to_drive(
        fc_all.filter(ee.Filter.gt('roi_coverage', 90)),
        fileFormat='CSV', 
        folder="export",
        description=f"rrs_samples_{name.replace(' ', '').lower()}_{id}", 
    )

In [ ]:
# fetch server-side data and convert to dataframe
data = fc_all.filter(ee.Filter.gt('roi_coverage', 90)).getInfo()
df_all = pd.DataFrame.from_dict(data['features'])
df_all = df_all.join(df_all["properties"].apply(pd.Series)).drop(columns=['geometry', 'properties'])
df_all['timestamp'] = pd.to_datetime(df_all['system:time_start'], unit='ms')

In [ ]:
# plot with hvplot
df_all['red_median'] = np.where(df_all['platform']=='LANDSAT-7', df_all['B3_median'], df_all['B4_median'])
df_all['nir_median'] = np.where(df_all['platform']=='LANDSAT-7', df_all['B4_median'], df_all['B5_median'])
df_all.hvplot.line(x='timestamp', y='red_median', groupby='station_id') *\
df_all.hvplot.scatter(x='timestamp', y='red_median', color='platform', groupby='station_id')

In [ ]:
# plot with hvplot
df_all.hvplot.line(x='timestamp', y='add_spm_B4_nechad_median', groupby='station_id') *\
df_all.hvplot.scatter(x='timestamp', y='add_spm_B4_nechad_median', color='platform', groupby='station_id')

## Get map over virtual station

In [ ]:
bounds = ee.Algorithms.GeometryConstructors.Point(gdf_dams.iloc[0].geometry.coords[0]).buffer(5000).bounds()

In [ ]:
%load_ext autoreload
%autoreload 2
import functions_process as funcs_process
import functions_turbidity as funcs_turb
import functions_sampling as funcs_sampling

# get Rrs imagecolls
ic_msi = funcs_process.load_rrs_imcoll(sensor='msi', start_date=start_date, end_date=end_date, bounds=bounds)
ic_oli = funcs_process.load_rrs_imcoll(sensor='oli', start_date=start_date, end_date=end_date, bounds=bounds)
ic_etm = funcs_process.load_rrs_imcoll(sensor='etm', start_date=start_date, end_date=end_date, bounds=bounds)
ic_all = ic_msi.merge(ic_oli).merge(ic_etm)

# comput tsm features
ic_all = ic_all \
    .map(funcs_turb.calc_spm_nechad) \
    .map(funcs_turb.calc_tur_nechad) \
    .map(funcs_turb.calc_tur_dogliotti) \
    .map(funcs_turb.calc_indices)

In [ ]:
crs = ic_msi.first().select(0).projection().crs()
scale = 30

# wxee convert to xarray
#ds_msi = ic_msi.select('B4').limit(25).wx.to_xarray(scale=scale, region=bounds)

# geemap export
geemap.ee_export_image_collection_to_drive(ee.ImageCollection(ic_msi).select('B4'), folder='export/oli', maxPixels=200000000, region=bounds, scale=30)

In [ ]:
# Export red bands to geotiffs
geemap.ee_export_image_collection_to_drive(ee.ImageCollection(imcoll_etm).select('B3'), folder='export/msi', maxPixels=200000000, region=bounds, scale=30)
geemap.ee_export_image_collection_to_drive(ee.ImageCollection(imcoll_oli).select('B4'), folder='export/oli', maxPixels=200000000, region=bounds, scale=30)
geemap.ee_export_image_collection_to_drive(ee.ImageCollection(imcoll_msi).select('B4'), folder='export/oli', maxPixels=200000000, region=bounds, scale=30)

In [ ]:
ee.Algorithms.GeometryConstructors.Point(gdf_dams.iloc[0].geometry.coords[0]).buffer(5000).bounds()

In [ ]:
ds_msi.is_cloud.isel(time=0).plot()

In [ ]:
Map.centerObject(img_oli, 14)

In [ ]:
img_etm = ic_etm.sort('CLOUD_COVER', True).first()
img_oli = ic_oli.sort('CLOUD_COVER', True).first()
img_msi = ic_msi.sort('CLOUDY_PIXEL_PERCENTAGE', True).first()

#vis = {'bands': 'Rrs_B4', 'min': 0, 'max': 0.2}
Map.addLayer(ee.Image(img_etm),  {}, 'ETM+, Rrs_B3')
Map.addLayer(ee.Image(img_oli),  {}, 'OLI, Rrs_B4')
Map.addLayer(ee.Image(img_msi),  {}, 'MSI, Rrs_B4')
Map.centerObject(img_msi, 14)